# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
data_df = pd.read_csv('../WeatherPy/output_data/cities_data.csv')
locations = data_df[['Latitude','Longitude']]
hum = data_df['Humidity'].astype(float)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=hum, dissipating=False,
                                  max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
narrow_df = data_df[data_df['Max Temp']<80] 
narrow_df = narrow_df[data_df['Max Temp']>70]  
narrow_df = narrow_df[data_df['Wind Speed']<10] 
narrow_df = narrow_df[data_df['Cloudiness']==0]
narrow_df

C:\Users\javie\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\javie\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\javie\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,City,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
198,chilca,-76.7372,-12.5211,73.00,78,0,6.91,PE,1612462160
205,tatawin,10.4518,32.9297,71.60,20,0,4.61,TN,1612462162
206,ancud,-73.8203,-41.8697,79.00,53,0,9.22,CL,1612462162
286,bahia honda,-83.1592,22.9036,72.86,43,0,6.22,CU,1612462175
476,riyadh,46.7219,24.6877,71.60,23,0,6.91,SA,1612461922
525,qandala,49.8728,11.4720,74.57,62,0,7.70,SO,1612462218


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = narrow_df
hotel_df['Hotel Name']=0
hotel_df

,City,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
198,chilca,-76.7372,-12.5211,73.00,78,0,6.91,PE,1612462160,0
205,tatawin,10.4518,32.9297,71.60,20,0,4.61,TN,1612462162,0
206,ancud,-73.8203,-41.8697,79.00,53,0,9.22,CL,1612462162,0
286,bahia honda,-83.1592,22.9036,72.86,43,0,6.22,CU,1612462175,0
476,riyadh,46.7219,24.6877,71.60,23,0,6.91,SA,1612461922,0
525,qandala,49.8728,11.4720,74.57,62,0,7.70,SO,1612462218,0


In [10]:
locations = hotel_df[["Latitude", "Longitude"]]

target_radius = 5000
target_type = "lodging"

hotel_names=[]
city_name=[]
country_name=[]
lat=[]
lon=[]

for index, row in hotel_df.iterrows():
    target_coordinates = str(row['Latitude']) + ',' + str(row['Longitude'])

    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "types": target_type,
    "key": g_key}

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)

    places_data = response.json()
    
    try:
        hotel_name = places_data['results'][0]['name']
    
        hotel_names.append(hotel_name)
        city_name.append(row['City'])
        country_name.append(row['Country'])
        lat.append(row['Latitude'])
        lon.append(row['Longitude'])
    except:
        continue


In [11]:
hotel_df2 = pd.DataFrame({'Hotel Name':hotel_names,
                          'City':city_name,
                          'Country': country_name,
                          'Latitude':lat,
                          'Longitude':lon})
hotel_df2


,Hotel Name,City,Country,Latitude,Longitude
0,Playa Punta Yaya - Las Salinas,chilca,PE,-12.5211,-76.7372
1,Sangho Privilege Tataouine,tatawin,TN,32.9297,10.4518
2,Hotel Arena Gruesa,ancud,CL,-41.8697,-73.8203
3,Casa Gualberto - Deysi,bahia honda,CU,22.9036,-83.1592
4,InterContinental Riyadh,riyadh,SA,24.6877,46.7219


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df2.iterrows()]
locations = hotel_df2[["Latitude", "Longitude"]]

In [13]:

# Add marker layer ontop of heat map
marker_locations = locations

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))